# Quiz 6A: Feature Selection with Lasso Regression
Feature selection refers to the process of identifying which features are most important for justifying a given response. The emphasis is on identifying relevant features, not on minimizing error.

In this notebook, you will use LASSO to select features, using the solver provided by sklearn. Namely, you will:
* Do some minor feature engineering
* Verify experimentally that LASSO can be used for feature selection by setting some weights to 0
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set, with constraints on the size of selected features.

In [1]:
import numpy as np
import pandas as pd
np.set_printoptions(precision=2,suppress=True)

## Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = pd.read_csv('home_data.gz')
print(f'Input features:\n {sales.columns}')

Input features:
 Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')


## Create new features
Let's consider a few features that are transformations of input features.
* Squaring bedrooms will increase the separation between few bedrooms (e.g. 1) and lots of bedrooms (e.g. 4). Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living and sqft_lot will decrease the separation between big houses and small houses. The owner may not be exactly twice as happy for getting a house that is twice as big.

***Question 1.*** Add columns `sqft_living_sqrt`, `sqft_lot_sqrt`, `bedrooms_square`, and `floors_square`, which are the square root of `sqft_living` and `sqft_lot`, and the square of `bedrooms` and `floors`, respectively.

In [3]:
# YOUR CODE HERE
sales['sqft_living_sqrt'] = sales['sqft_living'] ** 1/2
sales['sqft_lot_sqrt'] = sales['sqft_lot'] ** 1/2
sales['bedrooms_square'] = sales['bedrooms'] ** 2
sales['floors_square'] = sales['floors'] ** 2
sales.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'sqft_living_sqrt',
       'sqft_lot_sqrt', 'bedrooms_square', 'floors_square'],
      dtype='object')

## Learn regression weights with L1 penalty
We start by fitting a model with most of the available features, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires using `Lasso` from `sklearn.linear_model`. Read the documentation before proceeding.

***Question 2.*** Create a Lasso model using `all_features` using 1E6 for L1 penalty.

In [5]:
# YOUR CODE HERE
from sklearn.linear_model import Lasso
clf = Lasso(alpha = 1e6)
clf.fit(sales[all_features], sales['price'])

Lasso(alpha=1000000.0)

By setting an L1 penalty that's large enough, we are selecting a subset of important features. 

***Question 3.*** Many of the weights should have been set to zero. Using code, find and report which features, including the intercept term, got non-zero weights. 

In [6]:
# YOUR CODE HERE
print(clf.coef_)
print(clf.intercept_)
#4 - sqft_living, 6 - sqft_lot, 14 - sqft_above, 16 - yr_built, 17 - yr_renovated

[  -0.     -0.      0.    285.78    0.     -0.29   -0.      0.      0.
    0.      0.      0.      0.      3.93    0.   -851.25   59.22]
1615810.0875070358


# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. 

***Question 4.*** Do a 3-way split of the sales data into train (45%), validation (45%), and test (10%) sets:
* Split our sales data into 2 sets: train+validation, and test
* Further split the train+validation data into two sets: training, validation
* Print the sizes of the three sets

*Make sure to use random_state=0 every time to sample a set to ensure consistent answers!*

In [7]:
# YOUR CODE HERE
training_and_validation = sales.sample(frac=0.9, random_state=0)
testing = sales.drop(training_and_validation.index)
training = training_and_validation.sample(frac=0.5, random_state=0)
validation = training_and_validation.drop(training.index)

testing = testing.reset_index(drop=True)
training = training.reset_index(drop=True)
validation = validation.reset_index(drop=True)

print(training.shape)
print(validation.shape)
print(testing.shape)

(9726, 25)
(9726, 25)
(2161, 25)


Next, we will fit several models with increasing values of L1 penalty.

***Question 5.*** 
* For `l1_penalty` values in $[10^2, 10^{2.5}, 10^3, 10^{3.5},\ldots, 10^9]$ (use `np.logspace`!) fit a regression model with a given `l1_penalty` on the training data. Specify `l1_penalty` in the argument of the constructor for the Lasso class.
* Compute the MSE on validation data (here you will want to use `.predict()`) for each `l1_penalty` and report the MSE and number of non-zero weights (*Hint: numpy has a useful function for this.*)

In [8]:
# YOUR CODE HERE
# sklearn will flag bad choices for the l1 penalty with a warning. Let's just ignore those
import warnings 
warnings.filterwarnings("ignore")

l1_penalty = np.logspace(2, 9, num = 15, base = 10, endpoint=True)

for i in l1_penalty:
    clf = Lasso(alpha = i)
    clf.fit(training[all_features], training['price'])
    y = validation['price']
    y_hat = clf.predict(validation[all_features])
    print(f'MSE: {np.dot(y - y_hat, y - y_hat)/len(y)}')
    print(f'Num of non-zero weights: {np.count_nonzero(clf.coef_)}')

MSE: 58528846674.020615
Num of non-zero weights: 15
MSE: 57444264397.03066
Num of non-zero weights: 15
MSE: 54797228276.04944
Num of non-zero weights: 15
MSE: 50978201427.56993
Num of non-zero weights: 13
MSE: 54116795207.66357
Num of non-zero weights: 11
MSE: 57977347130.133
Num of non-zero weights: 9
MSE: 66423047887.63249
Num of non-zero weights: 6
MSE: 65916531703.52077
Num of non-zero weights: 5
MSE: 67212564923.72041
Num of non-zero weights: 5
MSE: 69484654532.63876
Num of non-zero weights: 3
MSE: 69987915691.99457
Num of non-zero weights: 3
MSE: 71407663933.596
Num of non-zero weights: 2
MSE: 82084031225.06386
Num of non-zero weights: 2
MSE: 137323298345.5134
Num of non-zero weights: 1
MSE: 137752166371.79898
Num of non-zero weights: 1


***Question 6.*** What value of `l1_penalty` yielded the minimum error? Is choosing this minimum useful for feature selection? Explain.

10^3.5 yielded the lowest error. However, it isn't useful as it hasn't ruled out too many features (all weights except two are non-zero).

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 5 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that uses only a few features.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [9]:
target_nonzeros = 5

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a coarse range of possible `l1_penalty_values`.

***Question 7.*** Fit 20 log-spaced penalty values between 1E5 and 1E7 inclusive. For each penalty do the following:
* Fit a regression model with a given `l1_penalty` on training data. Specify `l1_penalty` as the argument for the constructor of Lasso.
* Extract the weights of the model and count the number of nonzeros (including the intercept term). Save the number of nonzeros to a list.

*Hint: Recall that `model.intercept_` and `model.coef_` give you lists with the parameters you learned.* 

In [10]:
# YOUR CODE HERE
l1_penalty_values = np.logspace(5, 7, num = 20, base = 10, endpoint=True)
l1_penalty_values
non_zeros = []

for i in l1_penalty_values:
    clf = Lasso(alpha = i)
    clf.fit(training[all_features], training['price'])
    nz = np.count_nonzero(clf.coef_)
    if clf.intercept_ != 0:
        nz = nz + 1
    non_zeros.append((i, nz))

non_zeros

[(100000.0, 7),
 (127427.49857031347, 7),
 (162377.67391887208, 7),
 (206913.808111479, 7),
 (263665.08987303555, 6),
 (335981.8286283781, 6),
 (428133.2398719396, 6),
 (545559.4781168514, 6),
 (695192.7961775606, 6),
 (885866.7904100833, 6),
 (1128837.8916846884, 5),
 (1438449.888287663, 6),
 (1832980.7108324375, 5),
 (2335721.4690901213, 5),
 (2976351.441631319, 5),
 (3792690.1907322537, 4),
 (4832930.238571752, 4),
 (6158482.110660254, 4),
 (7847599.703514607, 4),
 (10000000.0, 4)]

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros (too large a penalty), and at the other end, we will have `l1_penalty` values that have too many non-zeros (too small a penalty).  

More formally, programmatically find:
* The largest `l1_penalty` that has more non-zeros than `target_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `target_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)

***Question 8.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

In [11]:
# YOUR CODE HERE
l1_penalty_min_li = []
l1_penalty_max_li = []
for n in non_zeros:
    if n[1] > target_nonzeros:
        l1_penalty_min_li.append(n[0])
    if n[1] < target_nonzeros:
        l1_penalty_max_li.append(n[0])
        
l1_penalty_min = max(l1_penalty_min_li)
l1_penalty_max = min(l1_penalty_max_li)

print(l1_penalty_min, l1_penalty_max)

1438449.888287663 3792690.1907322537


## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [12]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
* Fit a regression model with a given `l1_penalty` on training data. Call Lasso with the argument `l1_penalty` to specify the penalty value.
* Measure the MSE of the learned model on the validation set

***Question 9.*** Find the model that the lowest MSE on the VALIDATION set and has sparsity *equal* to `target_nonzeros`.

In [13]:
# YOUR CODE HERE

MSE_li = []
RSS_li = []
nonz = []
coefs = []
pens = []

for i in l1_penalty_values:
    clf = Lasso(alpha = i)
    clf.fit(training[all_features], training['price'])
    y = validation['price']
    y_hat = clf.predict(validation[all_features])
    nz = np.count_nonzero(clf.coef_)
    if clf.intercept_ != 0:
        nz = nz + 1
    if nz == target_nonzeros:
        nonz.append(nz)
        MSE_li.append(np.dot(y - y_hat, y - y_hat)/len(y))
        RSS_li.append(np.dot(y - y_hat, y - y_hat))
        coefs.append((clf.coef_, clf.intercept_))
        pens.append(i)
        
min_mse = min(MSE_li)
print(min_mse)
model = coefs[MSE_li.index(min_mse)]
model

69458126365.41039


(array([ -0.  ,  -0.  ,   0.  , 281.78,   0.  ,  -0.18,  -0.  ,   0.  ,
          0.  ,   0.  ,   0.  ,   0.  ,   0.  ,  -0.  ,   6.41,  -0.  ,
         73.67]),
 -47444.0331156255)

***Question 10.***:
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the validation set and has sparsity *equal* to `target_nonzeros`?
2. What features in this model have non-zero coefficients?

In [14]:
# YOUR CODE HERE
min_rss = min(RSS_li)
min_pen = pens[RSS_li.index(min_rss)]
min_pen

1686264.6569660408

4 - sqft_living, 6 - sqft_lot, 15 - sqft_basement, 17 - yr_renovated